# rossman pharmaceuticals

## Contents
* Libraries
* Dataset
* Preprocessing
* Building models
* Hyperparameter tuning
* Post prediction analysis
* Serialize models

In [ ]:
#Import Needed libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from hyperopt import hp, tpe
from hyperopt.fmin import fmin
import pickle

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, QuantileTransformer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Load data
train = pd.read_csv("/content/drive/My Drive/rossmann-store-sales/train.csv", low_memory= False)
test = pd.read_csv("/content/drive/My Drive/rossmann-store-sales/test.csv", low_memory= False)

In [ ]:
#Sanity check
train['StateHoliday'].replace({0:'o','0':'o'},inplace=True)
test['StateHoliday'].replace({'0':'o'},inplace=True)

The data appears to be in most recent form aaccording to the date hece we sort

In [ ]:
#sort values
train = train.sort_values(by= 'Date', ascending= True).reset_index(drop = True)

In [ ]:
# target = train.Sales
# train = train.drop(['Sales', 'Customers'], axis = 1)

In [ ]:
train['Date'] = pd.to_datetime(train.Date)
test['Date'] = pd.to_datetime(test.Date)

In [ ]:
#Engineer features
train['year'] = train.Date.apply(lambda x: x.year )
train['month'] = train.Date.apply(lambda x: x.month)
train['dow'] = train.Date.apply(lambda x: x.dayofweek )
# train['day_name'] = train.Date.apply(lambda x: x.day_name() )
# train['month_name'] = train.Date.apply(lambda x: x.month_name() )
train['doy'] = train.Date.apply(lambda x: x.dayofyear )
train['quarter'] = train.Date.apply(lambda x: x.quarter )
train['month_start'] = train.Date.apply(lambda x: 1 if x.is_month_start else 0)
train['month_end'] = train.Date.apply(lambda x: 1 if x.is_month_end else 0 )
train['is_weekend'] = train.dow.apply(lambda x: 1 if x < 5 else 0)
train['dom'] = train.Date.apply(lambda x: x.day)
train['evenDays'] = train.dom.apply(lambda x: 0 if x%2 else 1)


test['year'] = test.Date.apply(lambda x: x.year )
test['month'] = test.Date.apply(lambda x: x.month)
test['dow'] = test.Date.apply(lambda x: x.dayofweek )
test['doy'] = test.Date.apply(lambda x: x.dayofyear )
test['quarter'] = test.Date.apply(lambda x: x.quarter )
test['month_start'] = test.Date.apply(lambda x: x.is_month_start )
test['month_end'] = test.Date.apply(lambda x: x.is_month_end )
test['is_weekend'] = test.dow.apply(lambda x: 1 if x < 5 else 0)
test['dom'] = test.Date.apply(lambda x: x.day)
test['evenDays'] = test.dom.apply(lambda x: 0 if x%2 else 1)

In [ ]:
test.drop(['Id','dom'],axis=1, inplace=True)

In [ ]:
train.drop(['dom'], axis = 1, inplace = True)

In [ ]:
train.shape, test.shape

((1017209, 18), (41088, 16))

In [ ]:
#split into train and test set
X_train = train[train.Date < '2015-05-01']
X_test = train[train.Date > '2015-04-30']

In [ ]:
X_train.shape, X_test.shape, test.shape

((914629, 18), (102580, 18), (41088, 16))

In [ ]:
y_train = X_train.Sales
y_test = X_test.Sales

In [ ]:
X_train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,year,month,dow,doy,quarter,month_start,month_end,is_weekend,evenDays
0,1115,2,2013-01-01,0,0,0,0,a,1,2013,1,1,1,1,1,0,1,0
1,379,2,2013-01-01,0,0,0,0,a,1,2013,1,1,1,1,1,0,1,0
2,378,2,2013-01-01,0,0,0,0,a,1,2013,1,1,1,1,1,0,1,0
3,377,2,2013-01-01,0,0,0,0,a,1,2013,1,1,1,1,1,0,1,0
4,376,2,2013-01-01,0,0,0,0,a,1,2013,1,1,1,1,1,0,1,0


In [ ]:
d = ['Sales', 'Customers', 'Date']
X_train.drop(d, axis=1, inplace=True)
X_test.drop(d, axis = 1, inplace=True)

Define preprocessing steps

In [ ]:
#Pipeline to transform numeric columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
  ('Quantile_norm', QuantileTransformer(output_distribution='normal'))])

#Pipeline to transform categorical columns
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [ ]:
#Get numeric columns
numeric_features = X_train._get_numeric_data().columns
categorical_features = X_train.select_dtypes(include=['object']).columns

In [ ]:
#Apply column transform
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

Import regressors

In [ ]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

In [ ]:
regressors = [
              ('LGBM', LGBMRegressor(use_best_model = True, n_estimators=1000)),
              ('XGB', XGBRegressor(n_estimators=1000, use_best_model =True)),
            ]

define some useful functions

In [ ]:
def rmse(y_true, y_pred):
  return mean_squared_error(y_true, y_pred)**0.5

def train_fit(X_train, X_test, y_train, y_test, regressor):

  #initialize model
  model = Pipeline(steps=[('preprocessor', preprocessor),
                      ('regressor', regressor)])
  
  #slice the pipeline to obtain the transform
  pipe_temp = Pipeline(model.steps[:-1])

  #apply the transform on train set
  X_trans = pipe_temp.fit_transform(X_train,y_train)

  #apply transform on eval set
  eval_set = [(X_trans, y_train), (pipe_temp.transform(X_test), y_test)]

  #append regressor
  pipe_temp.steps.append(model.steps[-1])

  #fit model
  pipe_temp.fit(X_train, y_train,
             regressor__eval_set = eval_set,
              regressor__eval_metric = 'rmse',
             regressor__early_stopping_rounds = 100,
              regressor__verbose= False)
  return pipe_temp

In [ ]:
f= {}
for label, regressor in regressors:

    pipe_temp = train_fit(X_train, X_test, y_train, y_test, regressor)
    
    print(label)
    print("Test score: %.3f" % rmse(y_test, pipe_temp.predict(X_test)))
    print("Train score: %.3f" % rmse(y_train, pipe_temp.predict(X_train)))
    print()
    f[label] = rmse(y_test, model.predict(X_test))

LGBM
Test score: 2367.780
Train score: 2324.348

[02:39:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGB
Test score: 2101.835
Train score: 2047.483



Hyperparameter tuning

In [ ]:
def objective_xgb(params):

    #set parameters to tune
    params = {
        'max_depth': int(params['max_depth']),
        'gamma': "{:.3f}".format(params['gamma']),
        'learning_rate': "{:.3f}".format(params['learning_rate']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
    }
    
    #fit model with parameters
    reg_xgb = Pipeline(steps=[('preprocessor', preprocessor),
                      ('regressor', XGBRegressor(random_state=23, **params, n_estimators=300))])
    #get cv score
    score = ( -1 * cross_val_score(reg_xgb, X_train, y_train, scoring='neg_root_mean_squared_error')).mean()
    print("error {:.3f} params {}".format(score, params))
    return score
#set space
space_xgb = {
    'max_depth': hp.quniform('max_depth', 2, 8, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0),
    'gamma': hp.uniform('gamma', 0.0, 0.5),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.5)
}

def objective_lgb(params):
    #set parameters to tune
    params = {
        'num_leaves': int(params['num_leaves']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
    }
    
    #fit model with parameters
    reg_lgb = Pipeline(steps=[('preprocessor', preprocessor),
                      ('regressor', LGBMRegressor(random_state=23, **params, n_estimators=5000))])
    #get cv score
    score = ( -1 * cross_val_score(reg_lgb, X_train, y_train, scoring='neg_root_mean_squared_error')).mean()
    print("error {:.3f} params {}".format(score, params))
    return score
#set space
space = {
    'num_leaves': hp.quniform('num_leaves', 8, 128, 2),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0),
   
}

In [ ]:
best_xgb = fmin(fn=objective_xgb,
            space=space_xgb,
            algo=tpe.suggest,
            max_evals=10)

[09:10:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:13:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:16:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:18:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:21:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
error 1827.469 params {'max_depth': 6, 'gamma': '0.095', 'learning_rate': '0.166', 'colsample_bytree': '0.313'}
[09:24:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:26:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

In [ ]:
best_lgb = fmin(fn=objective_lgb,
            space=space,
            algo=tpe.suggest,
            max_evals=10)

error 2274.898 params {'num_leaves': 104, 'colsample_bytree': '0.941'}
error 2256.098 params {'num_leaves': 54, 'colsample_bytree': '0.942'}
error 2256.727 params {'num_leaves': 66, 'colsample_bytree': '0.666'}
error 2273.219 params {'num_leaves': 126, 'colsample_bytree': '0.668'}
error 2242.708 params {'num_leaves': 22, 'colsample_bytree': '0.869'}
error 2260.408 params {'num_leaves': 120, 'colsample_bytree': '0.607'}
error 2270.510 params {'num_leaves': 112, 'colsample_bytree': '0.664'}
error 2265.565 params {'num_leaves': 114, 'colsample_bytree': '0.769'}
error 2257.416 params {'num_leaves': 88, 'colsample_bytree': '0.938'}
error 2267.470 params {'num_leaves': 108, 'colsample_bytree': '0.523'}
100%|██████████| 10/10 [5:52:06<00:00, 2112.62s/it, best loss: 2242.7081048988466]


In [ ]:
best_lgb

{'colsample_bytree': 0.8686865471599106, 'num_leaves': 22.0}

In [ ]:
best_xgb

{'colsample_bytree': 0.9140294253685348,
 'gamma': 0.14545586998361132,
 'learning_rate': 0.2678585631235746,
 'max_depth': 8.0}

In [ ]:
best_xgb['max_depth'] = 8; best_lgb['num_leaves'] = 22 ; best_xgb, best_lgb

({'colsample_bytree': 0.9140294253685348,
  'gamma': 0.14545586998361132,
  'learning_rate': 0.2678585631235746,
  'max_depth': 8},
 {'colsample_bytree': 0.8686865471599106, 'num_leaves': 22})

run the models with the optimal hyperparameter

In [ ]:
regressors_2 = [
              ('LGBM', LGBMRegressor(use_best_model = True, n_estimators=1000, random_state = 42, **best_lgb)),
              ('XGB', XGBRegressor(n_estimators=1000, use_best_model =True, random_state = 42, **best_xgb)),
            ]

In [ ]:
f_2= {}
for label, regressor in regressors_2:
    
    pipe_temp = train_fit(X_train, X_test, y_train, y_test)

    print(label)
    print("Test score: %.3f" % rmse(y_test, pipe_temp.predict(X_test)))
    print("Train score: %.3f" % rmse(y_train, pipe_temp.predict(X_train)))
    print()
    f_2[label] = rmse(y_test, model.predict(X_test))

LGBM
Test score: 2321.992
Train score: 2260.078

[12:03:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGB
Test score: 2050.959
Train score: 2007.944



pickle the XGBModel

In [ ]:
# filename = '/content/drive/My Drive/rossmann-store-sales/xgb_model.pkl'
# pickle.dump(pipe_temp, open(filename, 'wb'))

In [ ]:
regressors_2[1][1]

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9140294253685348,
             gamma=0.14545586998361132, importance_type='gain',
             learning_rate=0.2678585631235746, max_delta_step=0, max_depth=8,
             min_child_weight=1, missing=None, n_estimators=1000, n_jobs=1,
             nthread=None, objective='reg:linear', random_state=42, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
             subsample=1, use_best_model=True, verbosity=1)

Cross validation for the xgboost model

In [ ]:
score = []
y_pred_lgb = []

#init model
model = Pipeline(steps=[('preprocessor', preprocessor),
                      ('regressor', regressors_2[1][1])])
#set ts split
fold= TimeSeriesSplit(10)
for train_index, test_index in fold.split(X_train, y_train):

  #obtain data in fold
  X_t, X_val = X_train.iloc[train_index], X_train.iloc[test_index]
  y_t, y_val = y_train.iloc[train_index], y_train.iloc[test_index]
  
  #model
  pipe_temp = train_fit(X_t, X_val, y_t, y_val)

  #make_prediction  
  preds=pipe_temp.predict(X_val)

  #calculate score and prediction
  print("score: ",rmse(y_val,preds))
  score.append(rmse(y_val,preds))
  prob = pipe_temp.predict(X_test)
  y_pred_lgb.append(prob)


print('Interval: %.3f +/- %.3f' %(np.mean(score), 2*np.std(score)))
print('ScORE ON HOLDOUT SET: %.3f' %(rmse(y_test, np.mean(y_pred_lgb,0))))

[13:16:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
score:  1151.4311904198948
[13:17:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
score:  2032.1252595749297
[13:18:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
score:  2211.472661155893
[13:19:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
score:  2954.4952939698837
[13:20:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
score:  1965.0025872419124
[13:22:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
score:  2101.62367347624
[13:24:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear 

The confidence Interval of the prediction
> Interval: 2078.218 +/- 912.770

In [ ]:
imp = pd.DataFrame(regressors_2[1][1].feature_importance, index=X_train.columns, columns=['coef'])

In [ ]:
imp.sort_values(by = 'coef', ascending = False) ; imp

In [ ]:
##############Test pickle model##########

In [ ]:
class preprocess_data():

  def __init__(self, X):
    self.X = X

  def list2df(self):
    
    cols = ['Store', 'DayofWeek', 'Date', 'Open', 'Promo', 'StateHoliday','SchoolHoliday']
    self.X = np.array(self.X)
    df = pd.DataFrame(self.X, columns= cols)
    return df
  
  def feature_extraction(self):

    cols = ['Store', 'DayOfWeek', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday',
        'year', 'month', 'dow', 'doy', 'quarter', 'month_start', 'month_end',
        'is_weekend', 'evenDays']
        
    self.X = list2df(self.X)
    self.X['Date'] = pd.to_datetime(self.X['Date'])
    self.X['year'] = self.X.Date.apply(lambda x: x.year )
    self.X['month'] = self.X.Date.apply(lambda x: x.month)
    self.X['dow'] = self.X.Date.apply(lambda x: x.dayofweek )
    self.X['doy'] = self.X.Date.apply(lambda x: x.dayofyear )
    self.X['quarter'] = self.X.Date.apply(lambda x: x.quarter )
    self.X['month_start'] = self.X.Date.apply(lambda x: 1 if x.is_month_start else 0 )
    self.X['month_end'] = self.X.Date.apply(lambda x: 1 if x.is_month_end else 0)
    self.X['is_weekend'] = self.X.dow.apply(lambda x: 1 if x < 5 else 0)
    self.X['dom'] = self.X.Date.apply(lambda x: x.day)
    self.X['evenDays'] = self.X.dom.apply(lambda x: 0 if x%2 else 1)
    self.X.drop(['dom','Date'], axis = 1, inplace = True)
    self.X = pd.DataFrame(self.X.to_numpy(), columns=cols)

    return self.X

In [ ]:
data = [200, 3, '2013-03-02',  1, 1, 'a', 1]

In [ ]:
P = preprocess_data(data)
##data here is an example of what the user input will be

In [ ]:
d3 = P.feature_extraction()

In [ ]:
def predict_data(data):
  loaded_model = pickle.load(open(filename, 'rb'))
  result = loaded_model.predict(data)
  return ('The predicted sales is {}'.format(result))

In [ ]:
predict_data(d3)

[17:18:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


'The predicted sales is [6439.0293]'

In [ ]:
#pip freeze > requirements.txt